In [1]:
### CREATING RANDOMIZATION.CSV
import pandas as pd
import numpy as np
import random
import csv
mypath = ('C:/Users/luca.chech.16/Desktop/Random_probes_last/Luca/')

In [2]:
df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')
df.reset_index(drop=False)

image_names = df.index.tolist()
cat1 = df['0'].tolist()
cat2 = df['1'].tolist()
cat3 = df['2'].tolist()
cat4 = df['3'].tolist()
cat5 = df['4'].tolist()
ts = df['5'].tolist()
load = df['6'].tolist()
exemplar = df['7'].tolist()
possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}

In [3]:
# DEFINING TARGET-PRESENT PROBES
lista= possible_images_dict.keys()
target_present = []
intermediate = []
categories = [' bottle', ' horse', ' pottedplant', ' dog',
              ' cat', ' person', ' aeroplane', ' car', ' chair', ' sofa', ' bird', ' boat']
p_load = ['low','high']

for category in categories:
    for load in p_load:
        while True:
            trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
            if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                and trial not in intermediate and trial not in target_present:
                intermediate.append(trial)
            if len(intermediate) == 1:
                for each in intermediate:
                    target_present.append(each)
                    possible_images_dict[each].append(category)
                intermediate = []
                break

for n in target_present:
    possible_images_dict[n].append('Target Present')
# DEFINING TARGET-ABSENT PROBES
target_absent = []
intermediate = []
for category in categories:
    for load in p_load:
        while True:
            trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
            if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                and trial not in intermediate and trial not in target_absent and trial not in target_present:
                intermediate.append(trial)
            if len(intermediate) == 1:
                for each in intermediate:
                    target_absent.append(each)
                intermediate = []
                break
for each in target_absent:
    absent_list = [category for category in categories if category not in possible_images_dict[each]]
    absent_question = str(np.random.choice(absent_list, 1, replace= False))[2:-2]
    possible_images_dict[each].append(absent_question)
    possible_images_dict[each].append('Target Absent')
    print possible_images_dict[each]

for image in lista:
    if image not in target_present and image not in target_absent:
        possible_images_dict[image].append('NA')
        possible_images_dict[image].append('No Target')

# DEFINING CRITICAL TRIALS
critical_trials = []
def critical_trial(load_level):
    global critical_trials
    temp = []
    container = []
    for element in lista:
        if load_level in possible_images_dict[element]:
            container.append(element)
    for category in categories:
        while len(temp) <2 :
            intermediate = str(np.random.choice(container, 1, replace= False))[2:-2]
            if category in possible_images_dict[intermediate] \
                and intermediate not in temp and intermediate not in critical_trials:
                temp.append(intermediate)
        for each in temp:
            critical_trials.append(each)
        temp =[]
    print len(critical_trials)
         
critical_trial('low')
critical_trial('high')

print len(critical_trials) == len(set(critical_trials))
for each in critical_trials:
    possible_images_dict[each].append('Critical')
for each in possible_images_dict:
    if 'Critical' not in possible_images_dict[each]:
        possible_images_dict[each].append('Normal')

[' bicycle', ' bottle', nan, nan, nan, 18.893798014446219, 'low', ' bottle', ' cat', 'Target Absent']
[' person', ' bottle', nan, nan, nan, 41.29008547648025, 'high', ' bottle', ' cat', 'Target Absent']
[' horse', nan, nan, nan, nan, 19.234401336670953, 'low', ' horse', ' aeroplane', 'Target Absent']
[' person', ' horse', nan, nan, nan, 30.756714401720171, 'high', ' horse', ' bottle', 'Target Absent']
[' pottedplant', ' chair', ' diningtable', nan, nan, 21.053503723789198, 'low', ' pottedplant', ' car', 'Target Absent']
[' pottedplant', ' sofa', ' chair', nan, nan, 35.711398527657288, 'high', ' pottedplant', ' dog', 'Target Absent']
[' dog', nan, nan, nan, nan, 20.017034759943648, 'low', ' dog', ' pottedplant', 'Target Absent']
[' dog', nan, nan, nan, nan, 40.773121275105424, 'high', ' dog', ' cat', 'Target Absent']
[' cat', nan, nan, nan, nan, 13.685993646696661, 'low', ' cat', ' car', 'Target Absent']
[' person', ' cat', nan, nan, nan, 30.226516379371251, 'high', ' cat', ' pottedplan

In [4]:
#Defininfg blocks 
probes_per_block = []
critical_per_block = []
normal_per_block = []
present_used = []
absent_used = []
critical_used = []
normal_no_probe = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
normal_no_probe_used = []
block = []
to_insert = []

In [5]:
def block_low():
# Pick 3 present probes and 3 absent probes
    global critical_per_block,probes_per_block, present_used, absent_used, critical_used, normal_per_block, to_insert
    while len(probes_per_block) < 3:
        probe = str(np.random.choice(target_present,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in present_used and 'low' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    while 3 <= len(probes_per_block) < 6:
        probe = str(np.random.choice(target_absent,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in absent_used and 'low' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    for n in probes_per_block[0:3]:
        present_used.append(n)
    for n in probes_per_block[3:]:
        absent_used.append(n)
#------------------------------------------------------------------------------    
#Counting critical probes    
    critical_probes_counter = 0
    for n in probes_per_block:
        if 'Critical' in possible_images_dict[n]:
            critical_probes_counter = critical_probes_counter + 1
    #for n in probes_per_block:
    #    print possible_images_dict[n]
    print 'Present Used: ',len(present_used)
    print 'Absent Used ',len(absent_used)
    print 'Critical probes counter: ',critical_probes_counter
#------------------------------------------------------------------------------
#Picking critical trials
    while len(critical_per_block) < (6-critical_probes_counter):
        image = str(np.random.choice(critical_trials,1,replace = False))[2:-2]
        if image not in critical_per_block and image not in critical_used \
            and 'No Target' in possible_images_dict[image] and 'low' in possible_images_dict[image]:
            critical_per_block.append(image) 
    for n in critical_per_block:
        critical_used.append(n)
        print possible_images_dict[n]
    #temp = []
#------------------------------------------------------------------------------
#Picking normal trials
    while len(normal_per_block) < 54 - (6 - critical_probes_counter):
        image = str(np.random.choice(normal_no_probe,1,replace = False))[2:-2]
        if image not in normal_per_block and image not in normal_no_probe_used and 'low' in possible_images_dict[image]:
            normal_per_block.append(image)
    for n in normal_per_block:
        normal_no_probe_used.append(n)
    print 'len(normal_no_probe_used): ',len(normal_no_probe_used)
#-----------------------------------------------------------------------------
    to_insert = probes_per_block + critical_per_block
    print 'len(to_insert): ',len(to_insert)

    #Adding critical (no probe) trials to the block
    def constrained_sum_sample_pos_adding_criticals(n, total):
        """Return a randomly chosen list of n positive integers summing to total.
        Each such list is equally likely to occur."""
        global block, critical_per_block, new_list, diff, to_insert
        liste_scartate = 0
        while True:
            global sequences_length
            dividers = sorted(random.sample(xrange(1, total), n - 1))
            sequences_length = [a - b for a, b in zip(dividers + [total], [0] + dividers)]
            acc = 0
            new_list = []
            for i in sequences_length:
                acc = acc + i
                new_list.append(acc - 1)
            diff = []
            for i in range(len(to_insert)-1):
                diff.append(new_list[i+1] - new_list[i])
            liste_scartate = liste_scartate + 1
            if 1 not in diff and 2 not in diff:
                print 'Liste scartate: ',liste_scartate
                print 'New List: ', new_list
                block = normal_per_block
                #print len(block)
                random.shuffle(to_insert)
                for i in range(len(to_insert)):
                    block.insert(new_list[i], to_insert[i])
                new_list = [n for n in new_list if 'No Target' not in possible_images_dict[block[n]]]
                print new_list
                print len(new_list)
                return  block, new_list
                break
#-----------------------------------------------------------------------------    
    print 'len(block): ',len(block)
    constrained_sum_sample_pos_adding_criticals(len(to_insert),60)
    print 'len(block): ',len(block)
#print new_list
    print  
    for n in range(len(new_list)):
        print new_list[n], possible_images_dict[block[new_list[n]]]
#print diff
    probes_per_block = []
    critical_per_block = []
    normal_per_block = []
    to_insert = []
    print 'Normal used:',len(normal_no_probe_used)
    print'---------------------------------------------------'
    return block, new_list
    

In [6]:
low_one, lista_low_one = block_low()
low_two, lista_low_two = block_low()
low_three, lista_low_three = block_low()
low_four, lista_low_four = block_low()

Present Used:  3
Absent Used  3
Critical probes counter:  1
[' aeroplane', nan, nan, nan, nan, 11.640662335376794, 'low', ' aeroplane', 'NA', 'No Target', 'Critical']
[' dog', ' bottle', nan, nan, nan, 18.86070334209068, 'low', ' bottle', 'NA', 'No Target', 'Critical']
[' sofa', nan, nan, nan, nan, 17.055909373433501, 'low', ' sofa', 'NA', 'No Target', 'Critical']
[' sofa', nan, nan, nan, nan, 20.506510529107977, 'low', ' sofa', 'NA', 'No Target', 'Critical']
[' person', nan, nan, nan, nan, 18.879557532788805, 'low', ' person', 'NA', 'No Target', 'Critical']
len(normal_no_probe_used):  49
len(to_insert):  11
len(block):  0
Liste scartate:  79
New List:  [3, 6, 11, 15, 20, 34, 37, 41, 47, 55, 59]
[3, 6, 20, 34, 37, 55]
6
len(block):  60

3 [' chair', nan, nan, nan, nan, 20.138559565527519, 'low', ' chair', ' dog', 'Target Absent', 'Normal']
6 [' person', nan, nan, nan, nan, 19.276726337840987, 'low', ' person', ' person', 'Target Present', 'Normal']
20 [' cat', nan, nan, nan, nan, 13.68

In [7]:
def block_high():
# Pick 3 present probes and 3 absent probes
    global critical_per_block,probes_per_block, present_used, absent_used, critical_used, normal_per_block, to_insert
    while len(probes_per_block) < 3:
        probe = str(np.random.choice(target_present,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in present_used and 'high' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    while 3 <= len(probes_per_block) < 6:
        probe = str(np.random.choice(target_absent,1,replace = False))[2:-2]
        if probe not in probes_per_block and probe not in absent_used and 'high' in possible_images_dict[probe]:
            probes_per_block.append(probe)
    for n in probes_per_block[0:3]:
        present_used.append(n)
    for n in probes_per_block[3:]:
        absent_used.append(n)
#------------------------------------------------------------------------------    
#Counting critical probes    
    critical_probes_counter = 0
    for n in probes_per_block:
        if 'Critical' in possible_images_dict[n]:
            critical_probes_counter = critical_probes_counter + 1
    #for n in probes_per_block:
    #    print possible_images_dict[n]
    print 'Present Used: ',len(present_used)
    print 'Absent Used ',len(absent_used)
    print 'Critical probes counter: ',critical_probes_counter
#------------------------------------------------------------------------------
#Picking critical trials
    while len(critical_per_block) < (6-critical_probes_counter):
        image = str(np.random.choice(critical_trials,1,replace = False))[2:-2]
        if image not in critical_per_block and image not in critical_used \
            and 'No Target' in possible_images_dict[image] and 'high' in possible_images_dict[image]:
            critical_per_block.append(image) 
    for n in critical_per_block:
        critical_used.append(n)
        print possible_images_dict[n]
    #temp = []
#------------------------------------------------------------------------------
#Picking normal trials
    while len(normal_per_block) < 54 - (6 - critical_probes_counter):
        image = str(np.random.choice(normal_no_probe,1,replace = False))[2:-2]
        if image not in normal_per_block and image not in normal_no_probe_used and 'high' in possible_images_dict[image]:
            normal_per_block.append(image)
    for n in normal_per_block:
        normal_no_probe_used.append(n)
    print 'len(normal_no_probe_used): ',len(normal_no_probe_used)
#-----------------------------------------------------------------------------
    to_insert = probes_per_block + critical_per_block
    print 'len(to_insert): ',len(to_insert)

    #Adding critical (no probe) trials to the block
    def constrained_sum_sample_pos_adding_criticals(n, total):
        """Return a randomly chosen list of n positive integers summing to total.
        Each such list is equally likely to occur."""
        global block, critical_per_block, new_list, diff, to_insert
        liste_scartate = 0
        while True:
            global sequences_length
            dividers = sorted(random.sample(xrange(1, total), n - 1))
            sequences_length = [a - b for a, b in zip(dividers + [total], [0] + dividers)]
            acc = 0
            new_list = []
            for i in sequences_length:
                acc = acc + i
                new_list.append(acc - 1)
            diff = []
            for i in range(len(to_insert)-1):
                diff.append(new_list[i+1] - new_list[i])
            liste_scartate = liste_scartate + 1
            if 1 not in diff and 2 not in diff:
                print 'Liste scartate: ',liste_scartate
                print 'New List: ', new_list
                block = normal_per_block
                #print len(block)
                random.shuffle(to_insert)
                for i in range(len(to_insert)):
                    block.insert(new_list[i], to_insert[i])
                new_list = [n for n in new_list if 'No Target' not in possible_images_dict[block[n]]]
                print new_list
                print len(new_list)
                return  block, new_list
                break
#-----------------------------------------------------------------------------    
    print 'len(block): ',len(block)
    constrained_sum_sample_pos_adding_criticals(len(to_insert),60)
    print 'len(block): ',len(block)
#print new_list
    print  
    for n in range(len(new_list)):
        print new_list[n], possible_images_dict[block[new_list[n]]]
#print diff
    probes_per_block = []
    critical_per_block = []
    normal_per_block = []
    to_insert = []
    print 'Normal used:',len(normal_no_probe_used)
    print'---------------------------------------------------'
    return block, new_list
    

In [8]:
high_one, lista_high_one = block_high()
high_two, lista_high_two = block_high()
high_three, lista_high_three = block_high()
high_four, lista_high_four = block_high()

Present Used:  15
Absent Used  15
Critical probes counter:  0
[' chair', ' bottle', nan, nan, nan, 33.568151144703606, 'high', ' bottle', 'NA', 'No Target', 'Critical']
[' pottedplant', ' person', ' chair', ' motorbike', ' boat', 31.580489902106891, 'high', ' boat', 'NA', 'No Target', 'Critical']
[' sofa', ' person', nan, nan, nan, 31.463677231105994, 'high', ' sofa', 'NA', 'No Target', 'Critical']
[' pottedplant', ' person', ' bottle', ' diningtable', nan, 35.499543649564117, 'high', ' bottle', 'NA', 'No Target', 'Critical']
[' bird', ' boat', nan, nan, nan, 29.314233597971626, 'high', ' bird', 'NA', 'No Target', 'Critical']
[' sofa', ' bottle', ' person', nan, nan, 29.479716970441856, 'high', ' sofa', 'NA', 'No Target', 'Critical']
len(normal_no_probe_used):  244
len(to_insert):  12
len(block):  60
Liste scartate:  547
New List:  [2, 6, 9, 12, 17, 20, 26, 29, 37, 43, 47, 59]
[6, 12, 20, 29, 37, 43]
6
len(block):  60

6 [' pottedplant', ' sofa', ' chair', nan, nan, 35.711398527657288,

In [9]:
print len(normal_no_probe)
print len(normal_no_probe_used)
print len(normal_no_probe) == len(set(normal_no_probe_used))
print len(target_present)
print len(present_used)
print len(target_present) == len(set(present_used))
print len(target_absent)
print len(absent_used)
print len(target_absent) == len(set(absent_used))
print len(critical_trials)
print len(critical_used)
critical_probes_counter = 0
probes = target_absent + target_present
for probe in probes:
    if 'Target Present' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
        critical_probes_counter = critical_probes_counter +1
    if 'Target Absent' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
        critical_probes_counter = critical_probes_counter +1
    
print critical_probes_counter

390
390
True
24
24
True
24
24
True
48
42
6


In [10]:
df = pd.DataFrame.from_dict(possible_images_dict)
df= df.T
df.columns = ['Cat.1','Cat.2','Cat.3','Cat.4','Cat.5','TS','Load','exemplar','question','present_absent','trial_type']
df.TS = df.TS.astype('float')
df.to_csv(mypath + 'randomization_1.csv')

In [11]:
df = pd.DataFrame.from_csv(mypath+'all_images_2.csv')
df.reset_index(drop=False)

image_names = df.index.tolist()
cat1 = df['0'].tolist()
cat2 = df['1'].tolist()
cat3 = df['2'].tolist()
cat4 = df['3'].tolist()
cat5 = df['4'].tolist()
ts = df['5'].tolist()
load = df['6'].tolist()
exemplar = df['7'].tolist()
possible_images_dict ={z[0]:list(z[1:]) for z in zip(image_names,cat1,cat2,cat3,cat4,cat5,ts,load,exemplar)}

In [12]:
# DEFINING TARGET-PRESENT PROBES
lista= possible_images_dict.keys()
target_present = []
intermediate = []
categories = [' bottle', ' horse', ' pottedplant', ' dog',
              ' cat', ' person', ' aeroplane', ' car', ' chair', ' sofa', ' bird', ' boat']
p_load = ['low','high']

for category in categories:
    for load in p_load:
        while True:
            trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
            if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                and trial not in intermediate and trial not in target_present:
                intermediate.append(trial)
            if len(intermediate) == 1:
                for each in intermediate:
                    target_present.append(each)
                    possible_images_dict[each].append(category)
                intermediate = []
                break

for n in target_present:
    possible_images_dict[n].append('Target Present')
# DEFINING TARGET-ABSENT PROBES
target_absent = []
intermediate = []
for category in categories:
    for load in p_load:
        while True:
            trial = str(np.random.choice(lista, 1, replace= False))[2:-2]
            if category in possible_images_dict[trial][7] and load in possible_images_dict[trial] \
                and trial not in intermediate and trial not in target_absent and trial not in target_present:
                intermediate.append(trial)
            if len(intermediate) == 1:
                for each in intermediate:
                    target_absent.append(each)
                intermediate = []
                break
for each in target_absent:
    absent_list = [category for category in categories if category not in possible_images_dict[each]]
    absent_question = str(np.random.choice(absent_list, 1, replace= False))[2:-2]
    possible_images_dict[each].append(absent_question)
    possible_images_dict[each].append('Target Absent')
    print possible_images_dict[each]

for image in lista:
    if image not in target_present and image not in target_absent:
        possible_images_dict[image].append('NA')
        possible_images_dict[image].append('No Target')

# DEFINING CRITICAL TRIALS
critical_trials = []
def critical_trial(load_level):
    global critical_trials
    temp = []
    container = []
    for element in lista:
        if load_level in possible_images_dict[element]:
            container.append(element)
    for category in categories:
        while len(temp) <2 :
            intermediate = str(np.random.choice(container, 1, replace= False))[2:-2]
            if category in possible_images_dict[intermediate] \
                and intermediate not in temp and intermediate not in critical_trials:
                temp.append(intermediate)
        for each in temp:
            critical_trials.append(each)
        temp =[]
    print len(critical_trials)
         
critical_trial('low')
critical_trial('high')

print len(critical_trials) == len(set(critical_trials))
for each in critical_trials:
    possible_images_dict[each].append('Critical')
for each in possible_images_dict:
    if 'Critical' not in possible_images_dict[each]:
        possible_images_dict[each].append('Normal')

[' car', ' bottle', ' person', nan, nan, 19.558737427898237, 'low', ' bottle', ' boat', 'Target Absent']
[' chair', ' bottle', nan, nan, nan, 33.568151144703606, 'high', ' bottle', ' boat', 'Target Absent']
[' person', ' horse', nan, nan, nan, 20.405728055332247, 'low', ' horse', ' car', 'Target Absent']
[' person', ' horse', nan, nan, nan, 32.480531856762035, 'high', ' horse', ' bird', 'Target Absent']
[' pottedplant', ' sofa', nan, nan, nan, 18.96772267222304, 'low', ' pottedplant', ' person', 'Target Absent']
[' pottedplant', ' sofa', nan, nan, nan, 29.042488868120479, 'high', ' pottedplant', ' cat', 'Target Absent']
[' dog', nan, nan, nan, nan, 20.390973131670112, 'low', ' dog', ' bottle', 'Target Absent']
[' dog', nan, nan, nan, nan, 33.960677529937648, 'high', ' dog', ' pottedplant', 'Target Absent']
[' cat', nan, nan, nan, nan, 16.827520571430128, 'low', ' cat', ' chair', 'Target Absent']
[' tvmonitor', ' cat', nan, nan, nan, 33.319550308764342, 'high', ' cat', ' bottle', 'Targe

In [13]:
#Defininfg blocks 
probes_per_block = []
critical_per_block = []
normal_per_block = []
present_used = []
absent_used = []
critical_used = []
normal_no_probe = [n for n in lista if 'Normal' in possible_images_dict[n] and 'No Target' in possible_images_dict[n]]
normal_no_probe_used = []
block = []
to_insert = []

In [14]:
low_five, lista_low_five = block_low()
low_six, lista_low_six = block_low()
low_seven, lista_low_seven = block_low()
low_eight, lista_low_eight = block_low()

Present Used:  3
Absent Used  3
Critical probes counter:  0
[' pottedplant', ' sofa', ' chair', nan, nan, 18.511887801846186, 'low', ' sofa', 'NA', 'No Target', 'Critical']
[' chair', nan, nan, nan, nan, 13.727438736718454, 'low', ' chair', 'NA', 'No Target', 'Critical']
[' car', nan, nan, nan, nan, 19.759255998410111, 'low', ' car', 'NA', 'No Target', 'Critical']
[' person', ' boat', nan, nan, nan, 16.651242460530831, 'low', ' boat', 'NA', 'No Target', 'Critical']
[' car', nan, nan, nan, nan, 19.892352124442521, 'low', ' car', 'NA', 'No Target', 'Critical']
[' dog', nan, nan, nan, nan, 14.526602394583092, 'low', ' dog', 'NA', 'No Target', 'Critical']
len(normal_no_probe_used):  48
len(to_insert):  12
len(block):  0
Liste scartate:  954
New List:  [0, 3, 7, 10, 18, 21, 27, 35, 38, 43, 50, 59]
[0, 21, 27, 38, 43, 59]
6
len(block):  60

0 [' person', ' horse', nan, nan, nan, 20.405728055332247, 'low', ' horse', ' car', 'Target Absent', 'Normal']
21 [' car', nan, nan, nan, nan, 18.1003221

In [15]:
high_five, lista_high_five = block_high()
high_six, lista_high_six = block_high()
high_seven, lista_high_seven = block_high()
high_eight, lista_high_eight = block_high()

Present Used:  15
Absent Used  15
Critical probes counter:  1
[' person', ' dog', nan, nan, nan, 30.556102874749641, 'high', ' dog', 'NA', 'No Target', 'Critical']
[' cat', nan, nan, nan, nan, 32.660663951184659, 'high', ' cat', 'NA', 'No Target', 'Critical']
[' sofa', ' chair', ' diningtable', nan, nan, 35.492312020496826, 'high', ' sofa', 'NA', 'No Target', 'Critical']
[' person', ' horse', nan, nan, nan, 31.969634324354409, 'high', ' horse', 'NA', 'No Target', 'Critical']
[' person', ' dog', nan, nan, nan, 29.073434519731283, 'high', ' dog', 'NA', 'No Target', 'Critical']
len(normal_no_probe_used):  245
len(to_insert):  11
len(block):  60
Liste scartate:  51
New List:  [3, 13, 16, 20, 23, 29, 34, 37, 40, 56, 59]
[13, 16, 23, 29, 34, 59]
6
len(block):  60

13 [' person', ' bicycle', ' bottle', nan, nan, 30.128053840122085, 'high', ' bottle', ' bottle', 'Target Present', 'Normal']
16 [' aeroplane', nan, nan, nan, nan, 32.202764020748539, 'high', ' aeroplane', ' aeroplane', 'Target Pre

In [16]:
print len(normal_no_probe)
print len(normal_no_probe_used)
print len(normal_no_probe) == len(set(normal_no_probe_used))
print len(target_present)
print len(present_used)
print len(target_present) == len(set(present_used))
print len(target_absent)
print len(absent_used)
print len(target_absent) == len(set(absent_used))
print len(critical_trials)
print len(critical_used)
critical_probes_counter = 0
probes = target_absent + target_present
for probe in probes:
    if 'Target Present' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
        critical_probes_counter = critical_probes_counter +1
    if 'Target Absent' in possible_images_dict[probe] and 'Critical' in possible_images_dict[probe]:
        critical_probes_counter = critical_probes_counter +1
    
print critical_probes_counter

391
391
True
24
24
True
24
24
True
48
41
7


In [17]:
df = pd.DataFrame.from_dict(possible_images_dict)
df= df.T
df.columns = ['Cat.1','Cat.2','Cat.3','Cat.4','Cat.5','TS','Load','exemplar','question','present_absent','trial_type']
df.TS = df.TS.astype('float')
df.to_csv(mypath + 'randomization_2.csv')